<a href="https://colab.research.google.com/github/VamsiKrishnaMannava/myspark/blob/main/Pyspark_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=febd56cd2f257fd4d694234b2304844b0172bd037df5924659250b9528dfe2b6
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName(name="Test-1").getOrCreate()

Pys

# Scenario-1 multi delimiters

In [ ]:
from pyspark.sql.functions import split, col

In [ ]:
data = ["1,Alice\t30|New York","2,Jonny\t36|New Addington",]

In [ ]:
df = spark.createDataFrame(data, "string")
df.show()

In [ ]:
split_col = split(df['value'],',|\t|\|')

In [ ]:
df = df.withColumn('id',split_col.getItem(0)).withColumn('name',split_col.getItem(1)).withColumn('age',split_col.getItem(2)).withColumn('city', split_col.getItem(3))

In [ ]:
df.columns

In [ ]:
df.show()

In [ ]:
df.select('id', 'name', 'age', 'city').show()

# Scenario-2 find missing

In [ ]:
data2 = [(1,),(2,),(4,),(5,),(7,),(8,),(10,)]

df_numbers = spark.createDataFrame(data2,["Number"])

In [ ]:
df_numbers.show()

In [ ]:
full_range = spark.range(1,11).toDF("Number")

In [ ]:
full_range.show()

In [ ]:
missingNumber = full_range.join(df_numbers,"Number","left_anti")

In [ ]:
missingNumber.sort(["Number"]).show()

# Scenario-3 Top3 movies based on rating

In [ ]:
data_movies = [(1, "Movie A"),(2, "Movie B"),(3, "Movie C"),(4, "Movie D"),(5, "Movie E")]
columns_movies = ["MoviesID", "MovieName"]

In [ ]:
df_movies = spark.createDataFrame(data_movies, columns_movies)

In [ ]:
df_movies.sort("MoviesID").show()
df_movies.sort("MoviesID").take(3)

+--------+---------+
|MoviesID|MovieName|
+--------+---------+
|       1|  Movie A|
|       2|  Movie B|
|       3|  Movie C|
|       4|  Movie D|
|       5|  Movie E|
+--------+---------+



[Row(MoviesID=1, MovieName='Movie A'),
 Row(MoviesID=2, MovieName='Movie B'),
 Row(MoviesID=3, MovieName='Movie C')]

In [ ]:
data_ratings = [ (1, 101, 4.5), (1, 102, 4.0), (2, 103, 5.0),
                 (2, 104, 3.5), (3, 105, 4.0), (3, 106, 4.0),
                 (4, 107, 3.0), (5, 108, 2.5), (5, 109, 3.0)]

columns_ratings = ["MoviesID","UserID","Rating"]

df_ratings = spark.createDataFrame(data_ratings, columns_ratings)

df_ratings.sort("MoviesID").show()

+--------+------+------+
|MoviesID|UserID|Rating|
+--------+------+------+
|       1|   101|   4.5|
|       1|   102|   4.0|
|       2|   103|   5.0|
|       2|   104|   3.5|
|       3|   105|   4.0|
|       3|   106|   4.0|
|       4|   107|   3.0|
|       5|   108|   2.5|
|       5|   109|   3.0|
+--------+------+------+



In [ ]:
from pyspark.sql.functions import avg


In [ ]:
df_movies.createOrReplaceTempView("tab_movies")
df_ratings.createOrReplaceTempView("tab_ratings")

In [ ]:
query1 = "select * from tab_movies"
query2 = "select * from tab_ratings"

spark.sql(query1).show()
spark.sql(query2).show()

+--------+---------+
|MoviesID|MovieName|
+--------+---------+
|       1|  Movie A|
|       2|  Movie B|
|       3|  Movie C|
|       4|  Movie D|
|       5|  Movie E|
+--------+---------+

+--------+------+------+
|MoviesID|UserID|Rating|
+--------+------+------+
|       1|   101|   4.5|
|       1|   102|   4.0|
|       2|   103|   5.0|
|       2|   104|   3.5|
|       3|   105|   4.0|
|       3|   106|   4.0|
|       4|   107|   3.0|
|       5|   108|   2.5|
|       5|   109|   3.0|
+--------+------+------+



In [ ]:
query3 = """
select a.MoviesID, a.MovieName, avg(b.Rating) as Avg_Rating from tab_movies a, tab_ratings b where a.MoviesID = b.MoviesID group by a.MoviesID, a.MovieName
"""
spark.sql(query3).show()
spark.sql(query3).createOrReplaceTempView("total_ratings")

+--------+---------+----------+
|MoviesID|MovieName|Avg_Rating|
+--------+---------+----------+
|       1|  Movie A|      4.25|
|       2|  Movie B|      4.25|
|       3|  Movie C|       4.0|
|       4|  Movie D|       3.0|
|       5|  Movie E|      2.75|
+--------+---------+----------+



In [ ]:
query4 = """
select * from total_ratings order by Avg_Rating desc limit 3
"""
spark.sql(query4).show()

+--------+---------+----------+
|MoviesID|MovieName|Avg_Rating|
+--------+---------+----------+
|       1|  Movie A|      4.25|
|       2|  Movie B|      4.25|
|       3|  Movie C|       4.0|
+--------+---------+----------+



# Scenario-4 Rolling Avg past 7 days

In [ ]:
from pyspark.sql import Row
from pyspark.sql.window import Window
import pyspark.sql.functions as func

In [ ]:
salesdata = [
    Row(Date='2023-01-01', ProductID=100, QuantitySold=10),
    Row(Date='2023-01-02', ProductID=100, QuantitySold=15),
    Row(Date='2023-01-03', ProductID=100, QuantitySold=20),
    Row(Date='2023-01-04', ProductID=100, QuantitySold=25),
    Row(Date='2023-01-05', ProductID=100, QuantitySold=30),
    Row(Date='2023-01-06', ProductID=100, QuantitySold=35),
    Row(Date='2023-01-07', ProductID=100, QuantitySold=40),
    Row(Date='2023-01-08', ProductID=100, QuantitySold=45)
]

df_sales = spark.createDataFrame(salesdata)

df_sales.show()

+----------+---------+------------+
|      Date|ProductID|QuantitySold|
+----------+---------+------------+
|2023-01-01|      100|          10|
|2023-01-02|      100|          15|
|2023-01-03|      100|          20|
|2023-01-04|      100|          25|
|2023-01-05|      100|          30|
|2023-01-06|      100|          35|
|2023-01-07|      100|          40|
|2023-01-08|      100|          45|
+----------+---------+------------+



In [ ]:
df_sales = df_sales.withColumn("Date", func.to_date(func.col("Date")))

df_sales.show()

df_sales.printSchema()

+----------+---------+------------+
|      Date|ProductID|QuantitySold|
+----------+---------+------------+
|2023-01-01|      100|          10|
|2023-01-02|      100|          15|
|2023-01-03|      100|          20|
|2023-01-04|      100|          25|
|2023-01-05|      100|          30|
|2023-01-06|      100|          35|
|2023-01-07|      100|          40|
|2023-01-08|      100|          45|
+----------+---------+------------+

root
 |-- Date: date (nullable = true)
 |-- ProductID: long (nullable = true)
 |-- QuantitySold: long (nullable = true)



In [ ]:
windowsspec = Window.partitionBy('ProductID').orderBy('Date').rowsBetween(-6,0)

rollingavg = df_sales.withColumn('7DayAvg', func.avg('QuantitySold').over(windowsspec))
rollingavg.show()

+----------+---------+------------+-------+
|      Date|ProductID|QuantitySold|7DayAvg|
+----------+---------+------------+-------+
|2023-01-01|      100|          10|   10.0|
|2023-01-02|      100|          15|   12.5|
|2023-01-03|      100|          20|   15.0|
|2023-01-04|      100|          25|   17.5|
|2023-01-05|      100|          30|   20.0|
|2023-01-06|      100|          35|   22.5|
|2023-01-07|      100|          40|   25.0|
|2023-01-08|      100|          45|   30.0|
+----------+---------+------------+-------+



In [ ]:
# samething in sql

df_sales.createOrReplaceTempView("rollavg")
query5 = """
select
  a.*,
  avg(QuantitySold)
    over(partition by ProductID order by Date
        ROWS BETWEEN 6 PRECEDING AND CURRENT ROW) as 7dayavg
  from rollAVG a
"""

spark.sql(query5).show()

spark.sql(query5).printSchema()

+----------+---------+------------+-------+
|      Date|ProductID|QuantitySold|7dayavg|
+----------+---------+------------+-------+
|2023-01-01|      100|          10|   10.0|
|2023-01-02|      100|          15|   12.5|
|2023-01-03|      100|          20|   15.0|
|2023-01-04|      100|          25|   17.5|
|2023-01-05|      100|          30|   20.0|
|2023-01-06|      100|          35|   22.5|
|2023-01-07|      100|          40|   25.0|
|2023-01-08|      100|          45|   30.0|
+----------+---------+------------+-------+

root
 |-- Date: date (nullable = true)
 |-- ProductID: long (nullable = true)
 |-- QuantitySold: long (nullable = true)
 |-- 7dayavg: double (nullable = true)



# Scenario-5 : Catagorising data using UDFs

In [ ]:
from pyspark.sql import Row
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

In [ ]:
data3 = [
    Row(UserID=4001, Age=17),
    Row(UserID=4002, Age=45),
    Row(UserID=4003, Age=65),
    Row(UserID=4004, Age=30),
    Row(UserID=4005, Age=80)
]

df_cat = spark.createDataFrame(data3)

df_cat.show()

+------+---+
|UserID|Age|
+------+---+
|  4001| 17|
|  4002| 45|
|  4003| 65|
|  4004| 30|
|  4005| 80|
+------+---+



In [ ]:
def catagorize_age(age):
  if age < 18:
    return 'Youth'
  elif age < 60:
    return 'Adult'
  else:
    return 'Senior'

age_udf = udf(catagorize_age, StringType())

In [ ]:
df_cat = df_cat.withColumn("AgeGroup", age_udf(df_cat["Age"]))
df_cat.show(truncate=False)

+------+---+--------+
|UserID|Age|AgeGroup|
+------+---+--------+
|4001  |17 |Youth   |
|4002  |45 |Adult   |
|4003  |65 |Senior  |
|4004  |30 |Adult   |
|4005  |80 |Senior  |
+------+---+--------+



# Scenaroio-6 read json

In [ ]:
df_json = spark.read.json("/content/sample_data/anscombe.json")
df_json.show()

df_json.createOrReplaceTempView("jsondata")

query6="select Series, X,Y from jsondata where x is not null"
spark.sql(query6).show()

+------+----+-----+---------------+
|Series|   X|    Y|_corrupt_record|
+------+----+-----+---------------+
|  NULL|NULL| NULL|              [|
|     I|10.0| 8.04|           NULL|
|     I| 8.0| 6.95|           NULL|
|     I|13.0| 7.58|           NULL|
|     I| 9.0| 8.81|           NULL|
|     I|11.0| 8.33|           NULL|
|     I|14.0| 9.96|           NULL|
|     I| 6.0| 7.24|           NULL|
|     I| 4.0| 4.26|           NULL|
|     I|12.0|10.84|           NULL|
|     I| 7.0| 4.81|           NULL|
|     I| 5.0| 5.68|           NULL|
|    II|10.0| 9.14|           NULL|
|    II| 8.0| 8.14|           NULL|
|    II|13.0| 8.74|           NULL|
|    II| 9.0| 8.77|           NULL|
|    II|11.0| 9.26|           NULL|
|    II|14.0|  8.1|           NULL|
|    II| 6.0| 6.13|           NULL|
|    II| 4.0|  3.1|           NULL|
+------+----+-----+---------------+
only showing top 20 rows

+------+----+-----+
|Series|   X|    Y|
+------+----+-----+
|     I|10.0| 8.04|
|     I| 8.0| 6.95|
|     I|13

# Scenaroio-7 read csv

In [ ]:
df_csv = spark.read.csv("/content/sample_data/california_housing_test.csv", inferSchema=True, header=True)
df_csv.show()
df_csv.printSchema()

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+
|  -122.05|   37.37|              27.0|     3885.0|         661.0|    1537.0|     606.0|       6.6085|          344700.0|
|   -118.3|   34.26|              43.0|     1510.0|         310.0|     809.0|     277.0|        3.599|          176500.0|
|  -117.81|   33.78|              27.0|     3589.0|         507.0|    1484.0|     495.0|       5.7934|          270500.0|
|  -118.36|   33.82|              28.0|       67.0|          15.0|      49.0|      11.0|       6.1359|          330000.0|
|  -119.67|   36.33|              19.0|     1241.0|         244.0|     850.0|     237.0|       2.9375|           81700.0|
|  -119.56|   36.51|    

# Scenario-8 : first purchase date of each user


In [ ]:
from pyspark.sql import Row
import pyspark.sql.functions as f
from pyspark.sql.functions import min

In [ ]:
purchase_data = [
    Row(UserID=1, purchase_date='2023-01-05'),
    Row(UserID=1, purchase_date='2023-01-10'),
    Row(UserID=2, purchase_date='2023-01-03'),
    Row(UserID=3, purchase_date='2023-01-12')
]

df_purchase = spark.createDataFrame(purchase_data)

In [ ]:
df_purchase.show()

+------+-------------+
|UserID|purchase_date|
+------+-------------+
|     1|   2023-01-05|
|     1|   2023-01-10|
|     2|   2023-01-03|
|     3|   2023-01-12|
+------+-------------+



In [ ]:
df_purchase2 = df_purchase.withColumn("purchase_date", f.to_date(f.col("purchase_date")))
df_purchase2.createOrReplaceTempView("purchases")

In [ ]:
# using sql
query = "select UserID, min(purchase_date) from purchases group by UserId order by UserID"
spark.sql(query).show()

+------+------------------+
|UserID|min(purchase_date)|
+------+------------------+
|     1|        2023-01-05|
|     2|        2023-01-03|
|     3|        2023-01-12|
+------+------------------+



In [ ]:
df_firstpurchase = df_purchase2.groupBy("UserID").agg(min("purchase_date").alias("FirstPurchase_date")).orderBy("UserID")

df_firstpurchase.show()

+------+------------------+
|UserID|FirstPurchase_date|
+------+------------------+
|     1|        2023-01-05|
|     2|        2023-01-03|
|     3|        2023-01-12|
+------+------------------+



# Scenario-9: replace nulls with mean

In [75]:
from pyspark.sql.functions import mean, col

In [69]:
sales_data = [("1",100),("2",150),("3",None),("4",200),("5",None)]

df_sales = spark.createDataFrame(sales_data, ["sales_id","amount"])
df_sales.show()

+--------+------+
|sales_id|amount|
+--------+------+
|       1|   100|
|       2|   150|
|       3|  NULL|
|       4|   200|
|       5|  NULL|
+--------+------+



In [96]:
# sql way
df_sales.createOrReplaceTempView("meanreplace")
#query = "select sales_id, nvl(amount, mean(nvl(amount,0)) over (order by 1)) as meanamt from meanreplace"

query = """
with mntx as
  (select mean(amount) mnt from meanreplace where amount is not null)

select a.sales_id, nvl(a.amount, x.mnt) as meanamt from meanreplace a, mntx x
"""
spark.sql(query).show()  #answer

+--------+-------+
|sales_id|meanamt|
+--------+-------+
|       1|  100.0|
|       2|  150.0|
|       3|  150.0|
|       4|  200.0|
|       5|  150.0|
+--------+-------+



In [84]:
avg_value = df_sales.na.drop().agg(mean(col("amount")))
avg_value = avg_value.first()[0]

df_fullsales = df_sales.na.fill(avg_value)

df_fullsales.show()

+--------+------+
|sales_id|amount|
+--------+------+
|       1|   100|
|       2|   150|
|       3|   150|
|       4|   200|
|       5|   150|
+--------+------+



# Scenario-10: Split data by month


In [99]:
from pyspark.sql import Row

data = [
    Row("Product1", 100,150,200),
    Row("Product2", 200,250,300),
    Row("Product3", 300,350,400)
]

df_qtrdata = spark.createDataFrame(data, ["product","month1","month2","month3"])

df_qtrdata.show()

+--------+------+------+------+
| product|month1|month2|month3|
+--------+------+------+------+
|Product1|   100|   150|   200|
|Product2|   200|   250|   300|
|Product3|   300|   350|   400|
+--------+------+------+------+



In [102]:
df_qtrdata.selectExpr("product","stack(3, 'jan', month1, 'feb', month2, 'mar', month3)as (Month, sales)" ).show()

+--------+-----+-----+
| product|Month|sales|
+--------+-----+-----+
|Product1|  jan|  100|
|Product1|  feb|  150|
|Product1|  mar|  200|
|Product2|  jan|  200|
|Product2|  feb|  250|
|Product2|  mar|  300|
|Product3|  jan|  300|
|Product3|  feb|  350|
|Product3|  mar|  400|
+--------+-----+-----+

